In [1]:
from infrastructure.models import (
    CuratedProjectCollection,
    Project,
    ProjectStatus,
    Initiative,
    InfrastructureType,
    PowerPlant,
)
from locations.models import (
    LineStringGeometry,
    PointGeometry,
    PolygonGeometry,
    GeometryStore,
    Region,
    Country,
)
from django.db.models import (
    Case, CharField, Count, ExpressionWrapper, F, FloatField, Q, Value, When
)
from django.contrib.postgres.aggregates import StringAgg
from django.db.models.functions import Cast
from decimal import Decimal
from django.db.models.functions import Lower

In [2]:
PowerPlant.objects.count()

7203

In [3]:
Project.objects.count()

13889

In [4]:
Project.objects.filter(power_plant__isnull=True).count()

2836

In [5]:
Project.objects.filter(power_plant__isnull=False).count()

11053

In [6]:
GeometryStore.objects.count()

10470

In [7]:
# existing API queryset
existing_queryset = GeometryStore.objects.exclude(
    lines=None, points=None, polygons=None
).exclude(
    centroid__isnull=True
).filter(
    projects__isnull=False
).annotate(
    project_alt_name=F('projects__alternate_name'),
    project_name=F('projects__name'),
    # FYI this annotation is used in the filtering mechanism
    # If this is adjusted/removed, adjust the filter accordingly
    project_type=F('projects__infrastructure_type__name'),
    project_type_lower=Lower('projects__infrastructure_type__name'),
    locations=StringAgg('projects__countries__name', ',', distinct=True),
    currency=F('projects__total_cost_currency'),
    total_cost=F('projects__total_cost'),
).annotate(
    icon_image=Case(
        When(project_type_lower='seaport', then=Value('Seaport')),
        When(project_type_lower='dryport', then=Value('Dryport')),
        When(project_type_lower='rail', then=Value('Rail')),
        When(project_type_lower='road', then=Value('Road')),
        When(project_type_lower='multimodal', then=Value('Multimodal')),
        When(project_type_lower='intermodal', then=Value('Intermodal')),
        When(project_type_lower='powerplant', then=Value('Powerplant')),
        default=Value('dot'),
        output_field=CharField(),
    )
).annotate(
    best_project_name=Case(
        When(project_alt_name='', then=F('project_name')),
        default=F('project_alt_name'),
        output_field=CharField(),
    )
)
existing_queryset.count()

9358

In [8]:
projects_with_distinct_powerplants = Project.objects.exclude(
    power_plant__isnull=True
).exclude(
    geo__lines=None,
    geo__points=None,
    geo__polygons=None,
).exclude(
    geo__centroid__isnull=True
).order_by('power_plant__id').distinct('power_plant__id').only('id')
projects_with_distinct_powerplants.count()

5035

In [9]:
projects_without_powerplants = Project.objects.exclude(
    infrastructure_type__name='Powerplant',
).exclude(
    geo__lines=None,
    geo__points=None,
    geo__polygons=None,
).exclude(
    geo__centroid__isnull=True
).only('id')
projects_without_powerplants.count()

1466

In [10]:
5035 + 1466

6501

In [11]:
project_ids = []
project_ids.extend(list(projects_with_distinct_powerplants.values_list('id', flat=True)))
project_ids.extend(list(projects_without_powerplants.values_list('id', flat=True)))

# API queryset
queryset = GeometryStore.objects.exclude(
    lines=None, points=None, polygons=None
).exclude(
    centroid__isnull=True
).filter(
    projects__id__in=project_ids,  # <-- new filter
).annotate(
    project_alt_name=F('projects__alternate_name'),
    project_name=F('projects__name'),
    # FYI this annotation is used in the filtering mechanism
    # If this is adjusted/removed, adjust the filter accordingly
    project_type=F('projects__infrastructure_type__name'),
    project_type_lower=Lower('projects__infrastructure_type__name'),
    locations=StringAgg('projects__countries__name', ',', distinct=True),
    currency=F('projects__total_cost_currency'),
    total_cost=F('projects__total_cost'),
).annotate(
    icon_image=Case(
        When(project_type_lower='seaport', then=Value('Seaport')),
        When(project_type_lower='dryport', then=Value('Dryport')),
        When(project_type_lower='rail', then=Value('Rail')),
        When(project_type_lower='road', then=Value('Road')),
        When(project_type_lower='multimodal', then=Value('Multimodal')),
        When(project_type_lower='intermodal', then=Value('Intermodal')),
        When(project_type_lower='powerplant', then=Value('Powerplant')),
        default=Value('dot'),
        output_field=CharField(),
    )
).annotate(
    best_project_name=Case(
        When(project_alt_name='', then=F('project_name')),
        default=F('project_alt_name'),
        output_field=CharField(),
    )
)
queryset.count()

6501

In [12]:
pp = PowerPlant.objects.annotate(
    count=Count('project')
).filter(
    count__gt=1
).order_by('-count').values_list('name', 'count')
list(pp[:15])

[('Shandong Shidaowan Nuclear Power Plant', 19),
 ('Ananthapuramu Ultra Mega Solar PV Park', 15),
 ('Kirchheilingen', 14),
 ('Alto Minho I', 14),
 ('Parchim-UKA', 13),
 ('National Advanced PV Technology Demonstration Center Solar PV Park', 12),
 ('Vedanta Jharsuguda Captive Power Plant', 12),
 ('Yerkovetskaya Thermal Power Station', 12),
 ('Kudankulam Atomic Power Project', 12),
 ('Ukraine Nuclear Power Plant', 11),
 ('Gani - NTPC NSM Phase II Solar PV Park', 10),
 ('Tuapse Refinery Power Plant', 10),
 ('Amravati Thermal Power Plant', 10),
 ('Leningrad Nuclear Power Plant', 10),
 ('Kursk Nuclear Power Plant', 10)]

In [13]:
pp.count()

2084

In [14]:
# Non-powerplant projects
existing_queryset.exclude(projects__infrastructure_type__name='Powerplant').count()

1466

In [15]:
# existing queryset returns multiple
existing_queryset.filter(projects__power_plant__name='Shandong Shidaowan Nuclear Power Plant').count()

19

In [16]:
# only one Project should return with new queryset
queryset.filter(projects__power_plant__name='Shandong Shidaowan Nuclear Power Plant').count()

1

In [17]:
queryset.filter(projects__power_plant__name='Ananthapuramu Ultra Mega Solar PV Park').count()

1